# Quantity and Unit Conversions

In [1]:
# Add path to src/CARPy, in case notebook is running locally
import os, sys, warnings
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..\\..\\..\\src")))
warnings.simplefilter("ignore")  # <-- Suppress warnings

***
## Introduction

Engineers regularly have to deal with many types and systems of units.
CARPy simplifies working with different unit systems using:

1. `Quantity`: objects that capture the dimensionality of units, making the majority of unit conversions both robust and easy.

CARPy is built to support a wide array of units, but due to the inumerable possible combinations of units - proper testing of each unit conversion is not possible.
Users are encouraged to verify conversions are correct by hand, if in doubt.

***
## 1) `Quantity` objects

Start with relevant imports

In [2]:
from carpy.utility import Quantity

# Create a Quantity object
pressure_reading = Quantity(29.92, "inHg")

print(pressure_reading)

101320.748119 kg m⁻¹ s⁻²


`Quantity` objects display values in SI base units at every opportunity.
The inherent advantage of this representation is that mathematical operations and manipulations of the unit will always provide predictable values for the `Quantity` object.

For example, `pressure * area = force`:

In [3]:
wet_area = Quantity(4, "ft^{2}")

force = pressure_reading * wet_area
print(f"The wetted surface is experiencing a force of {force}")

The wetted surface is experiencing a force of 37652.02206132 kg m s⁻²


If you don't want to view units in base SI form, conversions are trivial:

In [4]:
# Convert quantity objects to other units using the '.to()' method!
print(f"{force} == {force.to('lbf')}, [lbf]")

37652.02206132 kg m s⁻² == 8464.511285172002, [lbf]


A full description of supported units is provided below (this table is best viewed in a wide screen):

In [5]:
import pandas as pd

from carpy.utility._unitconversion import dfs as units_dfs

# Properly implemented units will have a "transform" value (don't worry about what this means to you).
# We can filter out units that aren't yet fully defined to CARPy using this variable...
available_units = units_dfs["dimensions"][units_dfs["dimensions"]["transform"] != 0]
available_units = available_units.reset_index()

# For the units that are implemented show system, symbol(s), unit name(s), and related (SI base) dimensions
with pd.option_context(
    'display.max_rows', None,
   'display.max_columns', None,
   'display.width', 160
   ):
    print(available_units[["System", "Symbol(s)", "Name", "s", "m", "kg", "A", "K", "mol", "cd", "rad", "sr"]])

            System        Symbol(s)                                            Name   s   m  kg   A  K mol cd rad sr
0               SI                s                                          second   1   0   0   0  0   0  0   0  0
1               SI                m                                           metre   0   1   0   0  0   0  0   0  0
2               SI                g                                            gram   0   0   1   0  0   0  0   0  0
3               SI                A                                          ampere   0   0   0   1  0   0  0   0  0
4               SI                K                                          kelvin   0   0   0   0  1   0  0   0  0
5               SI              mol                                            mole   0   0   0   0  0   1  0   0  0
6               SI               cd                                         candela   0   0   0   0  0   0  1   0  0
7               SI              rad                             

As can be seen above, CARPy supports a number of conversions between systems such as SI, metric, imperial, U.S. customary, and other miscellaneous units.

You can currently even convert from denary to decibel scales:

In [6]:
# 30 decibel milliWatt
decibel_power = Quantity(30, "dBm")

print(f"30 dBm is equivalent to {decibel_power.to('dBW')} [dBW] or {decibel_power.to('W')} [W]")

30 dBm is equivalent to 0.0 [dBW] or 1.0 [W]


and of course, decibel units are also capable of manipulations like `power * time = energy` (thanks default SI representation!):

In [7]:
duration = Quantity(1, "s")

energy = decibel_power * duration
print(f"{decibel_power} acting over {duration} releases {energy}")

1. kg m² s⁻³ acting over 1. s releases 1. kg m² s⁻²


More complex, *compound* units can be specified too - just remember to put any powers of units inside of curly braces, and units should be separated by spaces:

In [8]:
# Thrust specific fuel consumption of an RD-843 liquid fuel rocket engine
tsfc_RD_843 = Quantity(323.2, "g kN^{-1} s^{-1}")

print(f"The RD-843 has a thrust specific fuel consumption of {tsfc_RD_843.to('lb lbf^{-1} hr^{-1}'):.2f} [lb/lbf/hr].")

The RD-843 has a thrust specific fuel consumption of 11.41 [lb/lbf/hr].
